# Práctica 4: Redes Neuronales Convolucionales

Importa las librerías que vayas a necesitar:

In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Seleccionar dataset e importarlo

Busca en la web de Kaggle (o en cualquier otra) un dataset de cualquier tema que te interese. Procura que no sea excesivamente grande para evitar un tiempo de entrenamiento mayor:

In [136]:
animals_train = tf.keras.utils.image_dataset_from_directory(directory="04_Data/animals151/dataset/", validation_split=0.2, subset="training", seed=413)
animals_val = tf.keras.utils.image_dataset_from_directory(directory="04_Data/animals151/dataset/", validation_split=0.2, subset="validation", seed=413)

Found 6270 files belonging to 151 classes.
Using 5016 files for training.
Found 6270 files belonging to 151 classes.
Using 1254 files for validation.


## Exploramos el dataset

Exploramos el formato del dataset: forma, número de elementos, número de "clases" finales...

In [137]:
animals_train.class_names

['acinonyx-jubatus',
 'aethia-cristatella',
 'agalychnis-callidryas',
 'agkistrodon-contortrix',
 'ailuropoda-melanoleuca',
 'ailurus-fulgens',
 'alces-alces',
 'anas-platyrhynchos',
 'ankylosaurus-magniventris',
 'apis-mellifera',
 'aptenodytes-forsteri',
 'aquila-chrysaetos',
 'ara-macao',
 'architeuthis-dux',
 'ardea-herodias',
 'balaenoptera-musculus',
 'betta-splendens',
 'bison-bison',
 'bos-gaurus',
 'bos-taurus',
 'bradypus-variegatus',
 'branta-canadensis',
 'canis-lupus',
 'canis-lupus-familiaris',
 'carcharodon-carcharias',
 'cardinalis-cardinalis',
 'cathartes-aura',
 'centrochelys-sulcata',
 'centruroides-vittatus',
 'ceratitis-capitata',
 'ceratotherium-simum',
 'chelonia-mydas',
 'chrysemys-picta',
 'circus-hudsonius',
 'codium-fragile',
 'coelacanthiformes',
 'colaptes-auratus',
 'connochaetes-gnou',
 'correlophus-ciliatus',
 'crocodylus-niloticus',
 'crotalus-atrox',
 'crotophaga-sulcirostris',
 'cryptoprocta-ferox',
 'cyanocitta-cristata',
 'danaus-plexippus',
 'dasyp

## Preprocesamiento del dataset

Haz las operaciones oportunas sobre el set de datos (en caso de ser necesarias). Por ejemplo, si se trata de imágenes en color y quieres trabajar en blanco y negro cambia sus dimensiones. O si son valores entre 0 y 255 escalalos al intervalo 0-1:

## Construir el Modelo

Construir la red neuronal requiere configurar las capas del modelo y luego compilar el modelo. Para ello, recuerda que puedes revisar el ejemplo realizado por Marcos y Carmen (Conv2D) o Ainhoa y Carlos (Conv3D) en la práctica 2, además de tener en cuenta los siguientes aspectos importantes:

*   Según nos adentramos en capas más profundas, el tamaño de las imágenes se debería ir reescalando por uno menor (utilizando la **capa MaxPool**)
*   Según nos adentramos en capas más profundas, la cantidad de capas convolucionales seguidas ha de ser mayor para que detecten patrones más complejos.
*   El final de la red será una **capa Flatten** para "aplanar" las imágenes resultantes de capas anteriores, seguido de tantas **capas Dense** como necesites.
*   Recuerda dejar en la última capa densa tantas neuronas como "clases" tengas que diferenciar.

In [138]:
num_classes = 151

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_classes)
])


### Compilamos el modelo

Antes de que el modelo este listo para entrenar , se necesitan algunas configuraciones mas: el optimizador, la función de coste o las métricas a utilizar:

In [139]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])


## Entrenar el Modelo

In [140]:
model.fit(
    animals_train,
    validation_data=animals_val,
    epochs=5
)

Epoch 1/5
157/157 [==============================] - 92s 582ms/step - loss: 4.8563 - accuracy: 0.0273 - val_loss: 4.5787 - val_accuracy: 0.0630
Epoch 2/5
157/157 [==============================] - 90s 574ms/step - loss: 3.6082 - accuracy: 0.2171 - val_loss: 4.4609 - val_accuracy: 0.1244
Epoch 3/5
157/157 [==============================] - 90s 576ms/step - loss: 1.5687 - accuracy: 0.6330 - val_loss: 5.4636 - val_accuracy: 0.1188
Epoch 4/5
157/157 [==============================] - 90s 574ms/step - loss: 0.5222 - accuracy: 0.8806 - val_loss: 7.6565 - val_accuracy: 0.1196
Epoch 5/5
157/157 [==============================] - 90s 575ms/step - loss: 0.2430 - accuracy: 0.9476 - val_loss: 9.1833 - val_accuracy: 0.1124


## Evaluar Exactitud

Comparar el rendimiento del modelo sobre el dataset de test:

In [141]:
model.evaluate(animals_val)

40/40 [==============================] - 6s 156ms/step - loss: 9.1833 - accuracy: 0.1124


[9.183300018310547, 0.11244019120931625]

## Hacer predicciones

Con el modelo entrenado probar a hacer predicciones sobre alguna imagen concreta: